# QUBO formulation of polynomial equation

In [1]:
from sympy.matrices import Matrix, SparseMatrix

# Use case

To illustrate the metohod we are taking the equation of the two node water system that reads:

$$
x_0^2 + x_1 - 3 = 0 \\
x_0 x_1 - 2 x_1^2 -1 = 0 
$$

In [2]:
import numpy as np

def nlfunc(input):
    x0,x1 = input

    def f0():
        return 2*x0**2 + 3*x0*x1 + x1**2 + 2*x0 + 4*x1 - 51
    
    def f1():
        return x0**2 + 2*x0*x1 + 2*x1**2 + 3*x0 + 2*x1 - 46
    
    
    return np.array([f0(), f1()])

## Classical Solution

The solution of such a small system can be obtained by newton raphson

In [3]:
from quantum_newton_raphson.newton_raphson import newton_raphson

initial_point = np.random.rand(2)
res = newton_raphson(nlfunc, initial_point)
assert np.allclose(nlfunc(res.solution), 0)

/home/nico/QuantumApplicationLab/QuantumNewtonRaphson/quantum_newton_raphson/utils.py:74: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn("spsolve requires A be CSC or CSR matrix format", SparseEfficiencyWarning)


In [4]:
ref_sol = res.solution
ref_sol

array([ 7.70957576, -3.52953964])

### Polynomial equation

We first write the polynomial equation as follow (https://www.nature.com/articles/s41598-019-46729-0) 

In [5]:
import numpy as np
import sparse
def define_matrices():
    
    # system of equations
    num_equations = 2
    num_variables = 2

    P0 = np.zeros((num_equations,1))
    P0[0] = -51
    P0[1] = -46

    P1 = np.zeros((num_equations, num_variables))
    P1[0, 0] = 2
    P1[0, 1] = 4
    P1[1, 0] = 3
    P1[1, 1] = 2


    P2 = np.zeros((num_equations, num_variables, num_variables))
    P2[0, 0, 0] = 2
    P2[0, 0, 1] = 3
    P2[0, 1, 1] = 1

    P2[1, 0, 0] = 1
    P2[1, 0, 1] = 2
    P2[1, 1, 1] = 2

    return P0, P1, P2 #, sparse.COO(P3)

matrices = define_matrices()

In [6]:
def verify_solution(x, matrices):
    """generates the classical solution."""

    P0, P1, P2 = matrices
    x = np.array([2,3]).reshape(-1,1)
    x2 = x@x.T
    return P0 + P1@x + [ [(P2[0] * x2).sum()],  [(P2[1] * x2).sum()] ]

verify_solution(np.array([2.,3]), matrices)

array([[0.],
       [0.]])

## 3. Solving the system

We will use here the `SimulatedAnnealingSampler` to be able to run that code locally. Quantum solvers are available through the Leap cloud service.

In [7]:
from qubols.qubo_poly import QUBO_POLY
from qubols.qubo_poly_mixed_variables import QUBO_POLY_MIXED
from qubols.encodings import PositiveQbitEncoding, RangedEfficientEncoding
from qubols.solution_vector import SolutionVector_V2 as SolutionVector

from dwave.samplers import SimulatedAnnealingSampler
from dwave.samplers import SteepestDescentSolver
from dwave.samplers import TabuSampler
from dimod import ExactSolver

nqbit = 4
step = 6/(2**nqbit-1)
step = 0.05
encoding = PositiveQbitEncoding(nqbit = nqbit, step=step, offset=0.0, var_base_name='x')
encoding = RangedEfficientEncoding(nqbit=nqbit, range=3, offset=3, var_base_name='x')
sol_vec = SolutionVector(2, encoding=encoding)
sampler = SimulatedAnnealingSampler()
# sampler = TabuSampler()

options = {'num_reads':1000, 'sampler':sampler}
qubo = QUBO_POLY(sol_vec, options)

In [8]:
np.sort(encoding.get_possible_values())

array([-4.28571429e-01,  1.66533454e-16,  4.28571429e-01,  8.57142857e-01,
        1.28571429e+00,  1.71428571e+00,  2.14285714e+00,  2.57142857e+00,
        3.00000000e+00,  3.42857143e+00,  3.85714286e+00,  4.28571429e+00,
        4.71428571e+00,  5.14285714e+00,  5.57142857e+00,  6.00000000e+00])

In [19]:
matrices = tuple(sparse.COO(m) for m in matrices)

bqm = qubo.create_bqm(matrices, strength=10000)

# sample
sampleset = qubo.sample_bqm(bqm, num_reads=1000)

# decode
sol  = qubo.decode_solution(sampleset.lowest().record[0][0])
sol = np.array(sol).reshape(-1)
elowest = sampleset.lowest().record[0][1]

ref_sol = [2, 3]
data_ref, eref = qubo.compute_energy(ref_sol, bqm)
data_sol, esol = qubo.compute_energy(sol, bqm)

print('ref: ', ref_sol, '->', data_ref[0], ' energy: ', eref)
print('sol: ', sol, '->', data_sol[0], ' energy: ', esol, elowest)
print(ref_sol - sol)

ref:  [2, 3] -> [2.142857142857143, 3.0]  energy:  -625.884214910453
sol:  [2.14285714 3.        ] -> [2.142857142857143, 3.0]  energy:  -625.884214910453 -625.8842149107077
[-0.14285714  0.        ]


In [20]:
qubo.verify_quadratic_constraints(sampleset.lowest())

In [11]:
import numpy as np
np.linalg.norm(nlfunc(data_ref[0]))

3.334034356383571

In [12]:
np.linalg.norm(nlfunc(data_sol[0]))

3.334034356383571

In [13]:
ref_sol = [2, 3]
trial_sol = [5.16, 1.33]
data_ref, eref = qubo.compute_energy(ref_sol, bqm)
data_sol, esol = qubo.compute_energy(trial_sol, bqm)

print('ref: ', ref_sol, '->', data_ref[0], ' energy: ', eref)
print('sol: ', trial_sol, '->', data_sol[0], ' energy: ', esol)
print(np.array(ref_sol) - np.array(trial_sol))

ref:  [2, 3] -> [2.142857142857143, 3.0]  energy:  -625.884214910453
sol:  [5.16, 1.33] -> [5.142857142857142, 1.2857142857142858]  energy:  1094.0953769262778
[-3.16  1.67]


In [14]:
sampleset.lowest().record[0][1]

-625.8842149107077

In [15]:
bqm.variables

Variables(['x_001_002', 'x_001_001', 'x_001_002*x_001_001', 'x_001_003', 'x_002_001', 'x_001_003*x_002_001', 'x_001_004', 'x_002_004', 'x_001_004*x_002_004', 'x_002_002', 'x_002_003', 'x_002_002*x_002_003', 'x_001_001*x_002_003', 'x_001_002*x_002_003', 'x_001_003*x_002_002', 'x_001_004*x_002_001', 'x_002_001*x_002_004', 'x_001_003*x_002_004', 'x_001_004*x_001_003', 'x_001_002*x_002_002', 'x_001_001*x_002_002', 'x_002_002*x_002_003*x_001_001', 'x_001_002*x_002_002*x_002_003', 'x_001_003*x_002_001*x_002_004', 'x_001_002*x_001_001*x_002_003', 'x_001_004*x_002_004*x_002_001', 'x_001_004*x_001_003*x_002_001', 'x_001_002*x_001_001*x_002_002', 'x_001_004*x_002_004*x_001_003', 'x_001_003*x_002_002*x_001_001', 'x_001_002*x_001_003*x_002_002'])

In [16]:
bqm.energy

<bound method BinaryQuadraticModel.energy of BinaryQuadraticModel({'x_001_002': 762.8334027488546, 'x_001_001': 1201.839233652644, 'x_001_002*x_001_001': 30000.0, 'x_001_003': 1903.771761765931, 'x_002_001': 1376.7663473552675, 'x_001_003*x_002_001': 30000.0, 'x_001_004': 5578.78550603915, 'x_002_004': 5490.0624739691775, 'x_001_004*x_002_004': 30000.0, 'x_002_002': 749.2328196584754, 'x_002_003': 1872.1099541857552, 'x_002_002*x_002_003': 30000.0, 'x_001_001*x_002_003': 30000.0, 'x_001_002*x_002_003': 30000.0, 'x_001_003*x_002_002': 30000.0, 'x_001_004*x_002_001': 30000.0, 'x_002_001*x_002_004': 30000.0, 'x_001_003*x_002_004': 30000.0, 'x_001_004*x_001_003': 30000.0, 'x_001_002*x_002_002': 30000.0, 'x_001_001*x_002_002': 30000.0, 'x_002_002*x_002_003*x_001_001': 30000.0, 'x_001_002*x_002_002*x_002_003': 30000.0, 'x_001_003*x_002_001*x_002_004': 30000.0, 'x_001_002*x_001_001*x_002_003': 30000.0, 'x_001_004*x_002_004*x_002_001': 30000.0, 'x_001_004*x_001_003*x_002_001': 30000.0, 'x_001_